In [1]:
import keras
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import SGD, Adam
from keras.models import Sequential,Input,Model,load_model
from keras.layers import Dense, Dropout, Flatten, GaussianNoise
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import TensorBoard
from keras import regularizers

Using TensorFlow backend.


In [2]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img

def preprocess_input_vgg(x):
    """Wrapper around keras.applications.vgg16.preprocess_input()
    to make it compatible for use with keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument.
    
    Parameters
    ----------
    x : a numpy 3darray (a single image to be preprocessed)
    
    Note we cannot pass keras.applications.vgg16.preprocess_input()
    directly to to keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument because the former expects a
    4D tensor whereas the latter expects a 3D tensor. Hence the
    existence of this wrapper.
    
    Returns a numpy 3darray (the preprocessed image).
    
    """
    from keras.applications.inception_v3 import preprocess_input
    X = np.expand_dims(x, axis=0)
    X = preprocess_input(X)
    return X[0]

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\binary dataset\vasc\train',
                                                    target_size=[140, 140],
                                                    batch_size=32,
                                                    class_mode='categorical')

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg)
validation_generator = validation_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\binary dataset\vasc\test',
                                                              target_size=[140, 140],
                                                              batch_size=32,
                                                              class_mode='categorical')

Found 569 images belonging to 2 classes.
Found 148 images belonging to 2 classes.


In [3]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model
inc_model = InceptionV3(weights='imagenet', include_top=False)

# get layers and add average pooling layer
x = inc_model.output
x = GlobalAveragePooling2D()(x)

# add fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)
# add output layer
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=inc_model.input, outputs=predictions)

# freeze pre-trained model area's layer
for layer in inc_model.layers:
    layer.trainable = False

# update the weight that are added
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'] )
model.fit_generator(train_generator,
                    steps_per_epoch=64,
                    nb_epoch=1,
                    validation_data=validation_generator,
                    validation_steps=32)

# choose the layers which are updated by training
#layer_num = len(model.layers)
#for layer in model.layers[:248]:
#    layer.trainable = False

for layer in model.layers:
    layer.trainable = True

# training
model.compile(optimizer=SGD(lr=1e-4, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

C:\Users\7\Anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:30: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\7\Anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=64, validation_data=<keras_pre..., validation_steps=32, epochs=1)`


Epoch 1/1
64/64 [==============================] - 48s 755ms/step - loss: 1.1853 - acc: 0.7574 - val_loss: 1.0644 - val_acc: 0.8141


In [4]:
history = model.fit_generator(train_generator,
                    steps_per_epoch=64,
                    nb_epoch=40,
                    shuffle = True,
                    validation_data=validation_generator,
                    validation_steps=16)

C:\Users\7\Anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
C:\Users\7\Anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=64, shuffle=True, validation_data=<keras_pre..., validation_steps=16, epochs=40)`
  


Epoch 1/40
64/64 [==============================] - 41s 640ms/step - loss: 0.5489 - acc: 0.8026 - val_loss: 0.4353 - val_acc: 0.8172
Epoch 2/40
64/64 [==============================] - 29s 450ms/step - loss: 0.3084 - acc: 0.8471 - val_loss: 0.3701 - val_acc: 0.8424
Epoch 3/40
64/64 [==============================] - 28s 439ms/step - loss: 0.2648 - acc: 0.8860 - val_loss: 0.3840 - val_acc: 0.8655
Epoch 4/40
64/64 [==============================] - 30s 470ms/step - loss: 0.2355 - acc: 0.9080 - val_loss: 0.3050 - val_acc: 0.9223
Epoch 5/40
64/64 [==============================] - 28s 436ms/step - loss: 0.2051 - acc: 0.9169 - val_loss: 0.2630 - val_acc: 0.9267
Epoch 6/40
64/64 [==============================] - 30s 468ms/step - loss: 0.1816 - acc: 0.9344 - val_loss: 0.2701 - val_acc: 0.9475
Epoch 7/40
64/64 [==============================] - 28s 444ms/step - loss: 0.1826 - acc: 0.9306 - val_loss: 0.2659 - val_acc: 0.9643
Epoch 8/40
64/64 [==============================] - 31s 487ms/step - 

In [5]:
import pandas as pd

predict_gen = validation_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\binary dataset\vasc\test',
                                                              target_size=[140,140],
                                                              batch_size=148,
                                                              class_mode='categorical')

X_val_sample, res = next(predict_gen)
y_pred = model.predict(X_val_sample)

CM = []
for i in range(2):
    CM.append([0, 0])

Found 148 images belonging to 2 classes.


In [6]:
for x, y in zip(res, y_pred):
    CM[np.argmax(x)][np.argmax(y)] = CM[np.argmax(x)][np.argmax(y)] + 1

for line in CM:
    print(line)

model.evaluate_generator(validation_generator, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)

[27, 1]
[0, 120]
5/5 [==============================] - 1s 249ms/step


[0.046140133579437796, 0.9932432432432432]

In [7]:
onedtrue = []
onedpred = []
for true, pred in zip(res, y_pred):
    onedtrue.append(np.argmax(true))
    onedpred.append(np.argmax(pred))

In [8]:
from sklearn.metrics import classification_report
target_names = ['bcc', 'bcc-non']
print(classification_report(onedtrue, onedpred, target_names=target_names))

              precision    recall  f1-score   support

         bcc       1.00      0.96      0.98        28
     bcc-non       0.99      1.00      1.00       120

   micro avg       0.99      0.99      0.99       148
   macro avg       1.00      0.98      0.99       148
weighted avg       0.99      0.99      0.99       148



In [9]:
model.save('nv-9932-40epc.h5')

# Ensemble

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import SGD, Adam
from keras.models import Sequential,Input,Model,load_model
from keras.layers import Dense, Dropout, Flatten, GaussianNoise
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import TensorBoard
from keras import regularizers


Using TensorFlow backend.


In [2]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img

def preprocess_input_vgg(x):
    """Wrapper around keras.applications.vgg16.preprocess_input()
    to make it compatible for use with keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument.
    
    Parameters
    ----------
    x : a numpy 3darray (a single image to be preprocessed)
    
    Note we cannot pass keras.applications.vgg16.preprocess_input()
    directly to to keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument because the former expects a
    4D tensor whereas the latter expects a 3D tensor. Hence the
    existence of this wrapper.
    
    Returns a numpy 3darray (the preprocessed image).
    
    """
    from keras.applications.inception_v3 import preprocess_input
    X = np.expand_dims(x, axis=0)
    X = preprocess_input(X)
    return X[0]

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\binary dataset\akiec\train',
                                                    target_size=[140, 140],
                                                    batch_size=32,
                                                    class_mode='categorical')

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg)
validation_generator = validation_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\binary dataset\akiec\test',
                                                              target_size=[140, 140],
                                                              batch_size=32,
                                                              class_mode='categorical')

Found 541 images belonging to 2 classes.
Found 136 images belonging to 2 classes.


In [3]:
model = load_model(r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\ensemble\vasc-9932-40epc.h5')

In [4]:
predictions = []
for k in range(506):
    predictions.append([])

In [5]:
import pandas as pd

predict_gen = validation_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\unbalanced\test',
                                                              target_size=[140,140],
                                                              batch_size=506,
                                                              shuffle = False,
                                                              class_mode='categorical')

X_val_sample, res = next(predict_gen)
y_pred = model.predict(X_val_sample)


Found 506 images belonging to 6 classes.


In [6]:
for index,y in enumerate(y_pred):
    predictions[index].append(y[0])

In [7]:
onedtrue = []
onedpred = []
for true, pred in zip(res, y_pred):
    onedtrue.append(np.argmax(true))
    onedpred.append(np.argmax(pred))

In [8]:
from sklearn.metrics import confusion_matrix
confusion_matrix(onedtrue, onedpred)

array([[  0,  66,   0,   0,   0,   0],
       [  0, 103,   0,   0,   0,   0],
       [  0, 103,   0,   0,   0,   0],
       [  1, 102,   0,   0,   0,   0],
       [  0, 103,   0,   0,   0,   0],
       [ 27,   1,   0,   0,   0,   0]], dtype=int64)

In [9]:
import pickle
with open("pred_vasc.txt", "wb") as fp:   #Pickling
    pickle.dump(predictions, fp)

In [10]:
predictions

[[0.0010408311],
 [0.0003917207],
 [0.00090792467],
 [0.0002816501],
 [0.0005247169],
 [1.7861585e-05],
 [0.00022702604],
 [5.769488e-05],
 [0.009355347],
 [1.5499447e-06],
 [0.058070935],
 [0.00016416135],
 [0.00015242015],
 [0.00033813735],
 [0.008894553],
 [0.0017075674],
 [0.0027401692],
 [0.010262178],
 [7.092925e-05],
 [0.00048571482],
 [0.00199676],
 [0.0054666707],
 [1.1720064e-05],
 [0.0016218718],
 [0.0044670943],
 [0.024800053],
 [0.00015845406],
 [0.004183255],
 [1.6259959e-05],
 [0.0062943143],
 [0.0009617379],
 [0.0009101497],
 [0.00028289232],
 [3.2384934e-05],
 [0.0023530517],
 [5.8868096e-05],
 [0.001232257],
 [0.00039036284],
 [3.8264454e-05],
 [0.0009396394],
 [0.016720783],
 [0.0015991911],
 [0.006201225],
 [0.0040766294],
 [8.463459e-05],
 [0.00019577827],
 [0.00089145574],
 [2.7188647e-05],
 [0.00066333055],
 [0.0024831034],
 [0.0030617253],
 [0.0008563358],
 [5.3416858e-05],
 [0.0016948734],
 [1.7351123e-05],
 [0.00057581544],
 [0.00018774426],
 [0.0017897945],
 